In [ ]:
!git  clone  https://huggingface.co/dangtr0408/StyleTTS2-lite-vi
%cd  StyleTTS2-lite-vi

Cloning into 'StyleTTS2-lite-vi'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 108 (delta 44), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (108/108), 6.21 MiB | 12.09 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
!pip  install  -r  requirements.txt

In [ ]:
from inference import StyleTTS2

import librosa
import IPython.display as ipd
import torch.cuda

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
config_path = "Models/config.yaml"
models_path = "Models/base_model_120k_vi.pth"

In [ ]:
speakers = {
    "id_1": {
        "path": "./reference_audio/vn_1.wav",   #Ref audio path
        "lang": "vi",                           #Default language
        "speed": 1.0,                           #Speaking speed
    },
    "id_2": {
        "path": "./reference_audio/vn_2.wav",
        "lang": "vi",
        "speed": 1.0,
    },
    "id_3": {
        "path": "./reference_audio/vn_3.wav",
        "lang": "vi",
        "speed": 1.0,
    },
    "id_4": {
        "path": "./reference_audio/vn_4.wav",
        "lang": "vi",
        "speed": 1.0,
    },
}
for id in speakers:
    max_samples = 24000*20 #max 20 seconds ref audio
    print(speakers[id]['path'])
    wave, sr = librosa.load(speakers[id]['path'], sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000:              audio = librosa.resample(audio, sr, 24000)
    if len(audio) > max_samples: audio = audio[:max_samples]
    display(ipd.Audio(audio, rate=24000, normalize=True))

In [ ]:
model             = StyleTTS2(config_path, models_path).eval().to(device)
default_speaker   = "[id_1]"  #STR    Default speaker used when no speaker_id is provided in the input
avg_style         = True      #BOOL   Split the ref audio and calculate the avg styles.
stabilize         = True      #BOOL   Stabilize speaking speed.
denoise           = 0.6       #FLOAT  Adjust the strength of the denoiser. Value range is [0, 1]
n_merge           = 18        #INT    Avoid short sentences by merging when a sentence has fewer than n words

In [ ]:
!sudo apt-get install espeak-ng

In [ ]:
text = """
[id_1]Quãng thời gian đi học là một trong những ký ức đẹp nhất trong cuộc đời mỗi con người. Mỗi mùa khai giảng là một dấu mốc đặc biệt, luôn là dịp được nhiều sinh viên mong đợi. Hàng năm, cứ vào dịp tháng 9, 10 là sinh viên Đại học Bách khoa Hà Nội lại nô nức tham dự Lễ khai giảng, mang theo niềm vui, hi vọng về một năm học mới. Ngày khai giảng năm học mới luôn rất ý nghĩa và được mong đợi đối với các em tân sinh viên.

"""

In [ ]:
with torch.no_grad():
    styles = model.get_styles(speakers, denoise, avg_style)
    r = model.generate(text, styles, stabilize, n_merge, default_speaker)

print('Synthesized:')
display(ipd.Audio(r, rate=24000, normalize=True))

In [ ]:
# Cài đặt các thư viện cho server API
!pip install fastapi "uvicorn[standard]" pyngrok python-multipart --quiet

print("Cài đặt môi trường hoàn tất.")
print("-" * 50)


In [ ]:
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.responses import Response
from pydantic import BaseModel
from pyngrok import ngrok, conf
import threading
import os
import time
import torch
import soundfile as sf
from io import BytesIO


# --- CẤU HÌNH NGROK ---
NGROK_AUTHTOKEN = "2tWsdY4JT9nt2pWbQnQpEPN4hEp_4p8KQCHZ45JrDMpaRnKo4"
conf.get_default().auth_token = NGROK_AUTHTOKEN

# --- KHỞI TẠO FASTAPI APP ---
app = FastAPI()

# Định nghĩa cấu trúc dữ liệu cho request, thêm speaker_id
class TTSRequest(BaseModel):
    text: str
    speaker_id: int = 1 # Giọng đọc mặc định là 1

@app.get("/", summary="Endpoint gốc để kiểm tra server")
def read_root():
    return {"message": "Server VietVoice-TTS đang hoạt động!"}

@app.post("/tts", summary="Tổng hợp văn bản thành giọng nói")
async def text_to_speech(request: TTSRequest):
    """
    Nhận văn bản và ID giọng đọc, tạo ra audio và trả về trực tiếp.
    """
    try:
        print(f"Nhận yêu cầu: text='{request.text}', speaker_id={request.speaker_id}")

        # 1. Định dạng lại văn bản với ID giọng đọc
        formatted_text = f"[id_{request.speaker_id}]{request.text}"

        # 2. Gọi model để sinh audio (dữ liệu thô dạng NumPy array)
        with torch.no_grad():
            # Các tham số này có thể được tùy chỉnh nếu cần
            audio_data = model.generate(formatted_text, styles, stabilize, n_merge, default_speaker)

        # 3. Chuyển đổi dữ liệu audio thô thành file WAV trong bộ nhớ
        byte_io = BytesIO()
        sf.write(byte_io, audio_data, samplerate=24000, format='WAV')
        wav_bytes = byte_io.getvalue()

        # 4. Trả về dữ liệu WAV bytes trực tiếp trong response
        return Response(content=wav_bytes, media_type="audio/wav")

    except Exception as e:
        print(f"Lỗi trong quá trình tổng hợp: {e}")
        raise HTTPException(status_code=500, detail=str(e))

def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

ngrok.kill()
server_thread = threading.Thread(target=run_app)
server_thread.start()
time.sleep(5)
try:
    public_url = ngrok.connect(8000)
    print("\n" + "="*55)
    print("FastAPI TTS Server đã được khởi chạy thành công!")
    print(f"URL công khai: {public_url}")
    print("Gửi yêu cầu POST tới endpoint: " + f"{public_url}/tts")
    print('Ví dụ payload: {"text": "xin chào", "speaker_id": 1}')
    print("="*55)
except Exception as e:
    print(f"\n Lỗi khi kết nối ngrok: {e}")